In [47]:

from langchain.memory import ConversationSummaryBufferMemory, ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts.chat import ChatPromptTemplate, ChatMessagePromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

# 예시들
examples = [
    {"movie": "타이타닉", 
     "answer": "🎻🚢💔"},
    {"movie": "러브스토리", 
     "answer": "💑🎶😭"},
     
]

# LLM 설정
llm = ChatOpenAI(temperature=0.1)

# 대화 요약 기반 메모리
memory = ConversationBufferMemory(
    # llm=llm,
    # max_token_limit=300,
    return_messages=True,
)


example_prompt = ChatPromptTemplate.from_messages([
    ("human", "영화제목 {movie}를 3개의 이모티콘으로 표현하면?"),
    ("ai", "{answer}"),
])

# FewShot 프롬프트 템플릿
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 전체 Prompt 구성
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화제목을 보고 3개의 이모티콘으로 표현해주는 마법사이며, 사용자가 과거에 질문한 내용을 기억해서 대답하는 경우에는 자연어로 대답할 수도 있어요."),
    few_shot_prompt, 
    MessagesPlaceholder(variable_name="history"), 
    ("human", "{question}"),
])

# 메모리 불러오는 함수
def load_memory(_):
    return memory.load_memory_variables({})["history"]

# 체인 구성
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

# 호출 함수
def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})
    print(result.content)
       
    print("\n=== 저장된 메모리 요약 ===")
    print(memory.load_memory_variables({})["history"])


In [48]:
invoke_chain("기생충")

🏠🐜💰

=== 저장된 메모리 요약 ===
[HumanMessage(content='기생충', additional_kwargs={}, response_metadata={}), AIMessage(content='🏠🐜💰', additional_kwargs={}, response_metadata={})]


In [49]:
invoke_chain("대부")

🔫💰🇮🇹

=== 저장된 메모리 요약 ===
[HumanMessage(content='기생충', additional_kwargs={}, response_metadata={}), AIMessage(content='🏠🐜💰', additional_kwargs={}, response_metadata={}), HumanMessage(content='대부', additional_kwargs={}, response_metadata={}), AIMessage(content='🔫💰🇮🇹', additional_kwargs={}, response_metadata={})]


In [50]:
invoke_chain("내가 질문한 영화제목을 마지막 두개만 알려줘")

"대부"와 "기생충"입니다.

=== 저장된 메모리 요약 ===
[HumanMessage(content='기생충', additional_kwargs={}, response_metadata={}), AIMessage(content='🏠🐜💰', additional_kwargs={}, response_metadata={}), HumanMessage(content='대부', additional_kwargs={}, response_metadata={}), AIMessage(content='🔫💰🇮🇹', additional_kwargs={}, response_metadata={}), HumanMessage(content='내가 질문한 영화제목을 마지막 두개만 알려줘', additional_kwargs={}, response_metadata={}), AIMessage(content='"대부"와 "기생충"입니다.', additional_kwargs={}, response_metadata={})]
